In [1]:
import pandas as pd
import numpy as np
import time
from sklearn.ensemble import RandomForestClassifier
from aif360.metrics import ClassificationMetric
from aif360.datasets import StandardDataset

pip install 'aif360[LawSchoolGPA]'


In [2]:
raw_df=pd.read_csv("data.csv",index_col=0)
raw_df.drop_duplicates()
display(raw_df)

,Age,Accessibility,EdLevel,Employment,Gender,MentalHealth,MainBranch,YearsCode,YearsCodePro,Country,PreviousSalary,HaveWorkedWith,ComputerSkills,Employed
0,<35,No,Master,1,Man,No,Dev,7,4,Sweden,51552.0,C++;Python;Git;PostgreSQL,4,0
1,<35,No,Undergraduate,1,Man,No,Dev,12,5,Spain,46482.0,Bash/Shell;HTML/CSS;JavaScript;Node.js;SQL;Typ...,12,1
2,<35,No,Master,1,Man,No,Dev,15,6,Germany,77290.0,C;C++;Java;Perl;Ruby;Git;Ruby on Rails,7,0
3,<35,No,Undergraduate,1,Man,No,Dev,9,6,Canada,46135.0,Bash/Shell;HTML/CSS;JavaScript;PHP;Ruby;SQL;Gi...,13,0
4,>35,No,PhD,0,Man,No,NotDev,40,30,Singapore,160932.0,C++;Python,2,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
73457,<35,No,Undergraduate,1,Man,No,Dev,7,2,Germany,41058.0,C#;HTML/CSS;JavaScript;TypeScript;Docker;Kuber...,13,1
73458,>35,No,Undergraduate,1,Man,No,Dev,21,16,United States of America,115000.0,C#;HTML/CSS;Java;JavaScript;npm;ASP.NET Core ;...,11,1
73459,<35,No,Undergraduate,1,Man,No,Dev,4,3,Nigeria,57720.0,HTML/CSS;JavaScript;TypeScript;Docker;Express;...,12,1
73460,<35,Yes,Undergraduate,1,Man,Yes,Dev,5,1,United States of America,70000.0,C#;HTML/CSS;JavaScript;SQL;TypeScript;npm;Yarn...,15,1


In [3]:
man_df=raw_df.copy().loc[raw_df["Gender"]=="Man"].loc[:,['Gender','Employed']]
woman_df=raw_df.copy().loc[raw_df["Gender"]!="Man"].loc[:,['Gender','Employed']]

def get_percentage(value):
    return round(value,4)*100

print(f"The privillege gorups take up {get_percentage(man_df.shape[0]/raw_df.shape[0])}% samples")
print(f"The unprivillege gorups take up {get_percentage(woman_df.shape[0]/raw_df.shape[0])}% samples")

man_label=man_df["Employed"].value_counts()
print(f"The privillege groups have {get_percentage(man_label[1]/man_df.shape[0])}% positive label")
print(f"The privillege groups have {get_percentage(man_label[0]/man_df.shape[0])}% negative lablel")

women_label=woman_df["Employed"].value_counts()
print(f"The unprivillege groups have {get_percentage(women_label[1]/woman_df.shape[0])}% positive label")
print(f"The unprivillege groups have {get_percentage(women_label[0]/woman_df.shape[0])}% negative lablel")

The privillege gorups take up 93.34% samples
The unprivillege gorups take up 6.660000000000001% samples
The privillege groups have 54.08% positive label
The privillege groups have 45.92% negative lablel
The unprivillege groups have 47.25% positive label
The unprivillege groups have 52.75% negative lablel


In [ ]:
#Extract HaveWorkedWith column into several binary feature (Bag of Words Like)
computer_skill=pd.DataFrame()
feature_count=0
milestone=7346
startTime=time.time()
for index,row in raw_df.iterrows():
    hww_dict=dict()
    if(index==milestone):
        print(f"{milestone} entries have been examined,{round(time.time()-startTime)} seconds has passed since last report")
        startTime=time.time()
        milestone+=7346
    hww=row["HaveWorkedWith"]
    if hww is not np.nan:
        hww_list=str.split(hww,";")
        for ww in hww_list:
            column_name=ww
            if column_name not in computer_skill:
                computer_skill=pd.concat([computer_skill,pd.Series([np.nan]*index,name=column_name)],axis=1)
                feature_count+=1
            hww_dict[column_name]=1
        new_row=pd.DataFrame(hww_dict,index=[index])
    else:
        new_row=pd.Series([[np.nan]*feature_count],index=[index])
    computer_skill=pd.concat([computer_skill,new_row])
computer_skill=computer_skill.replace(np.nan,0)
display(computer_skill)
#Because It took too much time processing, I write it to local file so it can be later read from file
computer_skill.to_csv("computer_skill.csv")

In [ ]:
#get extrated individual computer skill read from file
computer_skill=pd.read_csv("computer_skill.csv",index_col=0)

In [3]:
def min_max_normalisation(series):
    return (series - series.min()) / (series.max() - series.min())

#Remove unworkable feature
rdf=raw_df.copy().drop("HaveWorkedWith",axis=1)
#normalize PreviousSalary feature
rdf["PreviousSalary"]=min_max_normalisation(rdf["PreviousSalary"])
#Create seperate dataframe to incorporate Computer Skill, However this dataset does not work. May come back later
#cs_rdf=pd.concat([rdf.drop(["Employed"],axis=1),computer_skill,rdf["Employed"]],axis=1)
display(rdf)
#display(cs_rdf)

,Age,Accessibility,EdLevel,Employment,Gender,MentalHealth,MainBranch,YearsCode,YearsCodePro,Country,PreviousSalary,ComputerSkills,Employed
0,<35,No,Master,1,Man,No,Dev,7,4,Sweden,0.230139,4,0
1,<35,No,Undergraduate,1,Man,No,Dev,12,5,Spain,0.207505,12,1
2,<35,No,Master,1,Man,No,Dev,15,6,Germany,0.345042,7,0
3,<35,No,Undergraduate,1,Man,No,Dev,9,6,Canada,0.205956,13,0
4,>35,No,PhD,0,Man,No,NotDev,40,30,Singapore,0.718445,2,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...
73457,<35,No,Undergraduate,1,Man,No,Dev,7,2,Germany,0.183291,13,1
73458,>35,No,Undergraduate,1,Man,No,Dev,21,16,United States of America,0.513391,11,1
73459,<35,No,Undergraduate,1,Man,No,Dev,4,3,Nigeria,0.257675,12,1
73460,<35,Yes,Undergraduate,1,Man,Yes,Dev,5,1,United States of America,0.312497,15,1


In [4]:
folds_index=np.array_split(rdf.sample(frac=1).index,5)
raw_dataset=StandardDataset(df=rdf, label_name="Employed",
                                    favorable_classes=[1],
                                    scores_name="",
                                    protected_attribute_names=['Gender'],
                                    privileged_classes=[["Man"]],
                                    categorical_features=['Age','Accessibility','EdLevel','MentalHealth', 'MainBranch',"Country"])



In [6]:
Cross_Validation_Column=["Cross Validate 1","Cross Validate 2","Cross Validate 3","Cross Validate 4","Cross Validate 5","Average"]
Accuracy_data=pd.DataFrame(columns=Cross_Validation_Column)
Precision_data=pd.DataFrame(columns=Cross_Validation_Column)
Recall_data=pd.DataFrame(columns=Cross_Validation_Column)
DisparateImpact_data=pd.DataFrame(columns=Cross_Validation_Column)
DisparateParity_data=pd.DataFrame(columns=Cross_Validation_Column)
EqualOpportunity_data=pd.DataFrame(columns=Cross_Validation_Column)
EqualizedOdds_data=pd.DataFrame(columns=Cross_Validation_Column)
#Read progress from local file if something happend that the jupyter kernel needs to be restarted (like fix bug from source library and reloaded it)
#merics_data=pd.read_csv("Benchmark_data.csv",index_col=0)

privileged_groups = [{"Gender":1}]
unprivileged_groups = [{"Gender":0}]

In [5]:
Cross_Validation_Column=["Cross Validate 1","Cross Validate 2","Cross Validate 3","Cross Validate 4","Cross Validate 5","Average"]

Accuracy_data=pd.read_csv("job_accuracy.csv",index_col=0)
display(Accuracy_data)
Precision_data=pd.read_csv("job_precision.csv",index_col=0)
display(Precision_data)
Recall_data=pd.read_csv("job_recall.csv",index_col=0)
display(Recall_data)
DisparateImpact_data=pd.read_csv("job_disparate_impact.csv",index_col=0)
display(DisparateImpact_data)
DisparateParity_data=pd.read_csv("job_disparate_parity.csv",index_col=0)
display(DisparateParity_data)
EqualOpportunity_data=pd.read_csv("job_equal_opportunity.csv",index_col=0)
display(EqualOpportunity_data)
EqualizedOdds_data=pd.read_csv("job_equalized_odds.csv",index_col=0)
display(EqualizedOdds_data)

privileged_groups = [{"Gender":1}]
unprivileged_groups = [{"Gender":0}]

,Cross Validate 1,Cross Validate 2,Cross Validate 3,Cross Validate 4,Cross Validate 5,Average
Baseline,0.777105,0.777105,0.772053,0.772802,0.775592,0.774931
AdversarialDebiase,0.784319,0.779419,0.783011,0.783283,0.784781,0.782963
GridSearchReduction(EO),0.777037,0.778670,0.773142,0.769466,0.775932,0.774850
GridSearchReduction(DP),0.775675,0.777241,0.772121,0.768582,0.776817,0.774087
Reweight,0.774042,0.777513,0.772529,0.771712,0.775660,0.774291
DisparateImpactRemover(0.5),0.776288,0.771388,0.775796,0.774912,0.776409,0.774959
DisparateImpactRemover(0.7),0.764786,0.767781,0.764225,0.752246,0.773959,0.764599
DisparateImpactRemover(1.0),0.765262,0.766419,0.764089,0.741968,0.771372,0.761822
CalibratedEqOddsPostprocessing,0.774587,0.777990,0.772733,0.771372,0.776613,0.774659
RejectOptionClassification(EO),0.773838,0.775607,0.774299,0.768718,0.774980,0.773488


,Cross Validate 1,Cross Validate 2,Cross Validate 3,Cross Validate 4,Cross Validate 5,Average
Baseline,0.783797,0.776963,0.779619,0.779036,0.779090,0.779701
AdversarialDebiase,0.792083,0.807346,0.782687,0.772675,0.797530,0.790464
GridSearchReduction(EO),0.784398,0.777656,0.778268,0.774267,0.776708,0.778260
GridSearchReduction(DP),0.783239,0.777086,0.778681,0.775833,0.778416,0.778651
Reweight,0.782807,0.778213,0.778776,0.777337,0.776263,0.778679
DisparateImpactRemover(0.5),0.792824,0.765541,0.791640,0.801422,0.777101,0.785706
DisparateImpactRemover(0.7),0.811450,0.764557,0.816914,0.834415,0.774976,0.800462
DisparateImpactRemover(1.0),0.817948,0.763799,0.819064,0.836636,0.773728,0.802235
CalibratedEqOddsPostprocessing,0.782882,0.777047,0.778378,0.777872,0.777995,0.778835
RejectOptionClassification(EO),0.780032,0.775082,0.779345,0.776354,0.776800,0.777523


,Cross Validate 1,Cross Validate 2,Cross Validate 3,Cross Validate 4,Cross Validate 5,Average
Baseline,0.808879,0.814469,0.800915,0.811292,0.808424,0.808796
AdversarialDebiase,0.812421,0.768374,0.823656,0.850816,0.799362,0.810926
GridSearchReduction(EO),0.807614,0.817286,0.806251,0.811669,0.813784,0.811321
GridSearchReduction(DP),0.806223,0.814597,0.802820,0.806399,0.812891,0.808586
Reweight,0.802808,0.813188,0.803710,0.811669,0.813912,0.809057
DisparateImpactRemover(0.5),0.790918,0.821511,0.789226,0.777792,0.814295,0.798748
DisparateImpactRemover(0.7),0.733241,0.813700,0.721636,0.677792,0.811870,0.751648
DisparateImpactRemover(1.0),0.725145,0.811524,0.718333,0.651568,0.807403,0.742795
CalibratedEqOddsPostprocessing,0.804073,0.816645,0.804980,0.809787,0.813146,0.809726
RejectOptionClassification(EO),0.807361,0.814085,0.807267,0.805772,0.811104,0.809118


,Cross Validate 1,Cross Validate 2,Cross Validate 3,Cross Validate 4,Cross Validate 5,Average
Baseline,0.868369,0.849506,0.836084,0.900317,0.871382,0.865132
AdversarialDebiase,0.936992,1.007966,0.902299,1.036675,1.045745,0.985935
GridSearchReduction(EO),0.862968,0.860570,0.875929,0.912733,0.902493,0.882939
GridSearchReduction(DP),0.880214,0.866750,0.882383,0.896506,0.911695,0.887510
Reweight,0.870386,0.854299,0.837965,0.892148,0.873800,0.865720
DisparateImpactRemover(0.5),0.945499,0.881053,0.918571,1.007550,0.922448,0.935024
DisparateImpactRemover(0.7),1.049234,0.892625,1.026410,1.241399,0.920677,1.026069
DisparateImpactRemover(1.0),1.071036,0.888531,1.051063,1.315805,0.936630,1.052613
CalibratedEqOddsPostprocessing,0.865229,0.864394,0.820395,0.889362,0.860733,0.860023
RejectOptionClassification(EO),0.856270,0.849658,0.842559,0.896002,0.867630,0.862424


,Cross Validate 1,Cross Validate 2,Cross Validate 3,Cross Validate 4,Cross Validate 5,Average
Baseline,-0.073760,-0.084710,-0.091192,-0.056697,-0.071764,-0.075625
AdversarialDebiase,-0.034925,0.004028,-0.055436,0.021853,0.024379,-0.008020
GridSearchReduction(EO),-0.076636,-0.078625,-0.069422,-0.049922,-0.054824,-0.065886
GridSearchReduction(DP),-0.066895,-0.074917,-0.065469,-0.058766,-0.049457,-0.063101
Reweight,-0.072166,-0.081726,-0.090546,-0.061540,-0.071140,-0.075423
DisparateImpactRemover(0.5),-0.029365,-0.068394,-0.043724,0.003973,-0.043553,-0.036213
DisparateImpactRemover(0.7),0.023858,-0.061184,0.012477,0.104662,-0.044542,0.007054
DisparateImpactRemover(1.0),0.033722,-0.063427,0.023912,0.130629,-0.035409,0.017885
CalibratedEqOddsPostprocessing,-0.075175,-0.076449,-0.100692,-0.062952,-0.078324,-0.078718
RejectOptionClassification(EO),-0.080844,-0.084789,-0.088278,-0.058969,-0.074339,-0.077444


,Cross Validate 1,Cross Validate 2,Cross Validate 3,Cross Validate 4,Cross Validate 5,Average
Baseline,-0.013407,-0.037173,-0.042153,-0.016031,-0.027150,-0.027183
AdversarialDebiase,0.039821,0.053543,0.004738,0.060352,0.075964,0.046884
GridSearchReduction(EO),-0.007675,-0.028568,-0.009942,-0.011963,-0.011252,-0.013880
GridSearchReduction(DP),0.006962,-0.028031,-0.013405,-0.035404,-0.017498,-0.017475
Reweight,-0.009131,-0.038132,-0.040383,-0.018666,-0.013785,-0.024019
DisparateImpactRemover(0.5),0.014500,-0.009862,-0.008458,0.030769,0.012184,0.007827
DisparateImpactRemover(0.7),0.093494,0.003074,0.056111,0.159463,0.007561,0.063941
DisparateImpactRemover(1.0),0.099927,-0.001572,0.073815,0.182884,0.019489,0.074908
CalibratedEqOddsPostprocessing,-0.012671,-0.020929,-0.051198,-0.023367,-0.039348,-0.029503
RejectOptionClassification(EO),-0.022752,-0.039084,-0.041789,-0.021331,-0.010809,-0.027153


,Cross Validate 1,Cross Validate 2,Cross Validate 3,Cross Validate 4,Cross Validate 5,Average
Baseline,-0.067385,-0.053033,-0.059067,-0.023696,-0.040905,-0.048817
AdversarialDebiase,-0.040384,0.035669,-0.030302,0.059121,0.051905,0.015202
GridSearchReduction(EO),-0.078422,-0.048766,-0.046326,-0.015344,-0.022995,-0.042371
GridSearchReduction(DP),-0.073622,-0.042601,-0.035992,-0.010670,-0.006905,-0.033958
Reweight,-0.068985,-0.046406,-0.059337,-0.030942,-0.051826,-0.051499
DisparateImpactRemover(0.5),-0.007137,-0.048189,0.002178,0.051529,-0.022523,-0.004828
DisparateImpactRemover(0.7),0.019955,-0.046930,0.049027,0.122136,-0.020882,0.024661
DisparateImpactRemover(1.0),0.033365,-0.047402,0.054939,0.148219,-0.014882,0.034848
CalibratedEqOddsPostprocessing,-0.071383,-0.051607,-0.068878,-0.029330,-0.042208,-0.052681
RejectOptionClassification(EO),-0.073130,-0.051913,-0.053330,-0.024184,-0.060725,-0.052656


In [6]:
def cross_validation(row_name,train_test_process):
    global Accuracy_data
    global Precision_data
    global Recall_data
    global DisparateImpact_data
    global DisparateParity_data
    global EqualOpportunity_data
    global EqualizedOdds_data
    accuracy_row=[]
    precision_row=[]
    recall_row=[]
    disparateImpcat_row=[]
    disparateParity_row=[]
    EqualOpportunity_row=[]
    EqualizedOdss_row=[]
    for fold in range(5):
        test_fold=folds_index[fold]
        train_index=pd.Index([])
        for f in folds_index:
            if f is not test_fold:
                train_index=train_index.append(f)
        train_set=raw_dataset.subset(train_index)
        test_set=raw_dataset.subset(test_fold)
        test_pred=train_test_process(train_set,test_set)
        metric=ClassificationMetric(test_set,test_pred,unprivileged_groups=unprivileged_groups,
                                                privileged_groups=privileged_groups)
        #Record Result
        accuracy_row.append(metric.accuracy())
        precision_row.append(metric.precision())
        recall_row.append(metric.recall())
        disparateImpcat_row.append(metric.disparate_impact())
        disparateParity_row.append(metric.statistical_parity_difference())
        EqualOpportunity_row.append(metric.true_positive_rate_difference())
        EqualizedOdss_row.append(metric.false_positive_rate_difference())
    accuracy_row.append(np.average(accuracy_row))
    precision_row.append(np.average(precision_row))
    recall_row.append(np.average(recall_row))
    disparateImpcat_row.append(np.average(disparateImpcat_row))
    disparateParity_row.append(np.average(disparateParity_row))
    EqualOpportunity_row.append(np.average(EqualOpportunity_row))
    EqualizedOdss_row.append(np.average(EqualizedOdss_row))
    #Record Cross Validation
    Accuracy_data=pd.concat([Accuracy_data,pd.DataFrame([accuracy_row],index=[row_name],columns=Cross_Validation_Column)])
    Accuracy_data.columns.name="Accuracy"
    display(Accuracy_data)
    Precision_data=pd.concat([Precision_data,pd.DataFrame([precision_row],index=[row_name],columns=Cross_Validation_Column)])
    Precision_data.columns.name="Precision"
    display(Precision_data)
    Recall_data=pd.concat([Recall_data,pd.DataFrame([recall_row],index=[row_name],columns=Cross_Validation_Column)])
    Recall_data.columns.name="Recall"
    display(Recall_data)
    DisparateImpact_data=pd.concat([DisparateImpact_data,pd.DataFrame([disparateImpcat_row],index=[row_name],columns=Cross_Validation_Column)])
    DisparateImpact_data.columns.name="Disparate Impact"
    display(DisparateImpact_data)
    DisparateParity_data=pd.concat([DisparateParity_data,pd.DataFrame([disparateParity_row],index=[row_name],columns=Cross_Validation_Column)])
    DisparateParity_data.columns.name="Disparate Parity"
    display(DisparateParity_data)
    EqualOpportunity_data=pd.concat([EqualOpportunity_data,pd.DataFrame([EqualOpportunity_row],index=[row_name],columns=Cross_Validation_Column)])
    EqualOpportunity_data.columns.name="Equal Opportuniy"
    display(EqualOpportunity_data)
    EqualizedOdds_data=pd.concat([EqualizedOdds_data,pd.DataFrame([EqualizedOdss_row],index=[row_name],columns=Cross_Validation_Column)])
    EqualizedOdds_data.columns.name="Equalized Odds"
    display(EqualizedOdds_data)

In [8]:
#Train RandomForest as Baseline model on original Datasets
def baseline_CV(train_set, test_set):
    #Training
    baseline=RandomForestClassifier()
    baseline.fit(train_set.features,train_set.labels.ravel())
    #Testing
    pred=baseline.predict(test_set.features)
    test_pred=test_set.copy(deepcopy=True)
    test_pred.labels=pred
    return test_pred

cross_validation("Baseline",baseline_CV)

Accuracy,Cross Validate 1,Cross Validate 2,Cross Validate 3,Cross Validate 4,Cross Validate 5,Average
Baseline,0.777105,0.777105,0.772053,0.772802,0.775592,0.774931


Precision,Cross Validate 1,Cross Validate 2,Cross Validate 3,Cross Validate 4,Cross Validate 5,Average
Baseline,0.783797,0.776963,0.779619,0.779036,0.77909,0.779701


Recall,Cross Validate 1,Cross Validate 2,Cross Validate 3,Cross Validate 4,Cross Validate 5,Average
Baseline,0.808879,0.814469,0.800915,0.811292,0.808424,0.808796


Disparate Impact,Cross Validate 1,Cross Validate 2,Cross Validate 3,Cross Validate 4,Cross Validate 5,Average
Baseline,0.868369,0.849506,0.836084,0.900317,0.871382,0.865132


Disparate Parity,Cross Validate 1,Cross Validate 2,Cross Validate 3,Cross Validate 4,Cross Validate 5,Average
Baseline,-0.07376,-0.08471,-0.091192,-0.056697,-0.071764,-0.075625


Equal Opportuniy,Cross Validate 1,Cross Validate 2,Cross Validate 3,Cross Validate 4,Cross Validate 5,Average
Baseline,-0.013407,-0.037173,-0.042153,-0.016031,-0.02715,-0.027183


Equalized Odds,Cross Validate 1,Cross Validate 2,Cross Validate 3,Cross Validate 4,Cross Validate 5,Average
Baseline,-0.067385,-0.053033,-0.059067,-0.023696,-0.040905,-0.048817


In [10]:
#Zhange et al. implementation
import tensorflow.compat.v1 as tf
from aif360.algorithms.inprocessing.adversarial_debiasing import AdversarialDebiasing
def AdversarialDebiase_CV(train_set,test_set):
    #Training
    tf.disable_eager_execution()
    tf.reset_default_graph()
    with tf.Session() as sess:
        debiased_model = AdversarialDebiasing(privileged_groups = privileged_groups,
                                unprivileged_groups = unprivileged_groups,
                                scope_name='debiased_classifier',
                                debias=True,
                                sess=sess)
        debiased_model.fit(train_set)
    #Testing
        test_pred=debiased_model.predict(test_set)
    return test_pred

cross_validation("AdversarialDebiase",AdversarialDebiase_CV)

Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.


epoch 0; iter: 0; batch classifier loss: 0.867668; batch adversarial loss: 1.349600
epoch 0; iter: 200; batch classifier loss: 1.105603; batch adversarial loss: 1.124752
epoch 0; iter: 400; batch classifier loss: 0.859138; batch adversarial loss: 0.854129
epoch 1; iter: 0; batch classifier loss: 0.951467; batch adversarial loss: 0.823189
epoch 1; iter: 200; batch classifier loss: 0.798129; batch adversarial loss: 0.589170
epoch 1; iter: 400; batch classifier loss: 0.690692; batch adversarial loss: 0.478141
epoch 2; iter: 0; batch classifier loss: 0.522942; batch adversarial loss: 0.434015
epoch 2; iter: 200; batch classifier loss: 0.563722; batch adversarial loss: 0.435651
epoch 2; iter: 400; batch classifier loss: 0.443536; batch adversarial loss: 0.359621
epoch 3; iter: 0; batch classifier loss: 0.505374; batch adversarial loss: 0.323851
epoch 3; iter: 200; batch classifier loss: 0.487278; batch adversarial loss: 0.320260
epoch 3; iter: 400; batch classifier loss: 0.466661; batch adv

Accuracy,Cross Validate 1,Cross Validate 2,Cross Validate 3,Cross Validate 4,Cross Validate 5,Average
Baseline,0.777105,0.777105,0.772053,0.772802,0.775592,0.774931
AdversarialDebiase,0.784319,0.779419,0.783011,0.783283,0.784781,0.782963


Precision,Cross Validate 1,Cross Validate 2,Cross Validate 3,Cross Validate 4,Cross Validate 5,Average
Baseline,0.783797,0.776963,0.779619,0.779036,0.77909,0.779701
AdversarialDebiase,0.792083,0.807346,0.782687,0.772675,0.79753,0.790464


Recall,Cross Validate 1,Cross Validate 2,Cross Validate 3,Cross Validate 4,Cross Validate 5,Average
Baseline,0.808879,0.814469,0.800915,0.811292,0.808424,0.808796
AdversarialDebiase,0.812421,0.768374,0.823656,0.850816,0.799362,0.810926


Disparate Impact,Cross Validate 1,Cross Validate 2,Cross Validate 3,Cross Validate 4,Cross Validate 5,Average
Baseline,0.868369,0.849506,0.836084,0.900317,0.871382,0.865132
AdversarialDebiase,0.936992,1.007966,0.902299,1.036675,1.045745,0.985935


Disparate Parity,Cross Validate 1,Cross Validate 2,Cross Validate 3,Cross Validate 4,Cross Validate 5,Average
Baseline,-0.073760,-0.084710,-0.091192,-0.056697,-0.071764,-0.075625
AdversarialDebiase,-0.034925,0.004028,-0.055436,0.021853,0.024379,-0.008020


Equal Opportuniy,Cross Validate 1,Cross Validate 2,Cross Validate 3,Cross Validate 4,Cross Validate 5,Average
Baseline,-0.013407,-0.037173,-0.042153,-0.016031,-0.027150,-0.027183
AdversarialDebiase,0.039821,0.053543,0.004738,0.060352,0.075964,0.046884


Equalized Odds,Cross Validate 1,Cross Validate 2,Cross Validate 3,Cross Validate 4,Cross Validate 5,Average
Baseline,-0.067385,-0.053033,-0.059067,-0.023696,-0.040905,-0.048817
AdversarialDebiase,-0.040384,0.035669,-0.030302,0.059121,0.051905,0.015202


In [12]:
#Agarwal et al., 2018, Agarwal et al., 2019 Implementation
from aif360.algorithms.inprocessing import GridSearchReduction
def GridSearchReduction_EO_CV(train_set,test_set):
    #Training
    gsr_eo_model=GridSearchReduction(estimator=RandomForestClassifier(),constraints="EqualizedOdds",prot_attr="Gender")
    gsr_eo_model.fit(train_set)
    #Testing
    test_pred=gsr_eo_model.predict(test_set)
    return test_pred

cross_validation("GridSearchReduction(EO)",GridSearchReduction_EO_CV)

c:\Users\15783\source\repos\Fairness-Metrics-Research\venv\Lib\site-packages\sklearn\preprocessing\_label.py:114: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
c:\Users\15783\source\repos\Fairness-Metrics-Research\venv\Lib\site-packages\sklearn\preprocessing\_label.py:114: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
c:\Users\15783\source\repos\Fairness-Metrics-Research\venv\Lib\site-packages\sklearn\preprocessing\_label.py:114: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
c:\Users\15783\source\repos\Fairness-Metrics-Research\venv\Lib\site-packages\sklearn\prepro

Accuracy,Cross Validate 1,Cross Validate 2,Cross Validate 3,Cross Validate 4,Cross Validate 5,Average
Baseline,0.777105,0.777105,0.772053,0.772802,0.775592,0.774931
AdversarialDebiase,0.784319,0.779419,0.783011,0.783283,0.784781,0.782963
GridSearchReduction(EO),0.777037,0.778670,0.773142,0.769466,0.775932,0.774850


Precision,Cross Validate 1,Cross Validate 2,Cross Validate 3,Cross Validate 4,Cross Validate 5,Average
Baseline,0.783797,0.776963,0.779619,0.779036,0.779090,0.779701
AdversarialDebiase,0.792083,0.807346,0.782687,0.772675,0.797530,0.790464
GridSearchReduction(EO),0.784398,0.777656,0.778268,0.774267,0.776708,0.778260


Recall,Cross Validate 1,Cross Validate 2,Cross Validate 3,Cross Validate 4,Cross Validate 5,Average
Baseline,0.808879,0.814469,0.800915,0.811292,0.808424,0.808796
AdversarialDebiase,0.812421,0.768374,0.823656,0.850816,0.799362,0.810926
GridSearchReduction(EO),0.807614,0.817286,0.806251,0.811669,0.813784,0.811321


Disparate Impact,Cross Validate 1,Cross Validate 2,Cross Validate 3,Cross Validate 4,Cross Validate 5,Average
Baseline,0.868369,0.849506,0.836084,0.900317,0.871382,0.865132
AdversarialDebiase,0.936992,1.007966,0.902299,1.036675,1.045745,0.985935
GridSearchReduction(EO),0.862968,0.860570,0.875929,0.912733,0.902493,0.882939


Disparate Parity,Cross Validate 1,Cross Validate 2,Cross Validate 3,Cross Validate 4,Cross Validate 5,Average
Baseline,-0.073760,-0.084710,-0.091192,-0.056697,-0.071764,-0.075625
AdversarialDebiase,-0.034925,0.004028,-0.055436,0.021853,0.024379,-0.008020
GridSearchReduction(EO),-0.076636,-0.078625,-0.069422,-0.049922,-0.054824,-0.065886


Equal Opportuniy,Cross Validate 1,Cross Validate 2,Cross Validate 3,Cross Validate 4,Cross Validate 5,Average
Baseline,-0.013407,-0.037173,-0.042153,-0.016031,-0.027150,-0.027183
AdversarialDebiase,0.039821,0.053543,0.004738,0.060352,0.075964,0.046884
GridSearchReduction(EO),-0.007675,-0.028568,-0.009942,-0.011963,-0.011252,-0.013880


Equalized Odds,Cross Validate 1,Cross Validate 2,Cross Validate 3,Cross Validate 4,Cross Validate 5,Average
Baseline,-0.067385,-0.053033,-0.059067,-0.023696,-0.040905,-0.048817
AdversarialDebiase,-0.040384,0.035669,-0.030302,0.059121,0.051905,0.015202
GridSearchReduction(EO),-0.078422,-0.048766,-0.046326,-0.015344,-0.022995,-0.042371


In [14]:
#Agarwal et al., 2018, Agarwal et al., 2019 Implementation

def GridSearchReduction_DP_CV(train_set,test_set):
    #Training
    gsr_dp_model=GridSearchReduction(estimator=RandomForestClassifier(),constraints="DemographicParity",prot_attr="Gender")
    gsr_dp_model.fit(train_set)
    #Testing
    test_pred=gsr_dp_model.predict(test_set)
    return test_pred

cross_validation("GridSearchReduction(DP)",GridSearchReduction_DP_CV)

c:\Users\15783\source\repos\Fairness-Metrics-Research\venv\Lib\site-packages\sklearn\preprocessing\_label.py:114: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
c:\Users\15783\source\repos\Fairness-Metrics-Research\venv\Lib\site-packages\sklearn\preprocessing\_label.py:114: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
c:\Users\15783\source\repos\Fairness-Metrics-Research\venv\Lib\site-packages\sklearn\preprocessing\_label.py:114: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
c:\Users\15783\source\repos\Fairness-Metrics-Research\venv\Lib\site-packages\sklearn\prepro

Accuracy,Cross Validate 1,Cross Validate 2,Cross Validate 3,Cross Validate 4,Cross Validate 5,Average
Baseline,0.777105,0.777105,0.772053,0.772802,0.775592,0.774931
AdversarialDebiase,0.784319,0.779419,0.783011,0.783283,0.784781,0.782963
GridSearchReduction(EO),0.777037,0.778670,0.773142,0.769466,0.775932,0.774850
GridSearchReduction(DP),0.775675,0.777241,0.772121,0.768582,0.776817,0.774087


Precision,Cross Validate 1,Cross Validate 2,Cross Validate 3,Cross Validate 4,Cross Validate 5,Average
Baseline,0.783797,0.776963,0.779619,0.779036,0.779090,0.779701
AdversarialDebiase,0.792083,0.807346,0.782687,0.772675,0.797530,0.790464
GridSearchReduction(EO),0.784398,0.777656,0.778268,0.774267,0.776708,0.778260
GridSearchReduction(DP),0.783239,0.777086,0.778681,0.775833,0.778416,0.778651


Recall,Cross Validate 1,Cross Validate 2,Cross Validate 3,Cross Validate 4,Cross Validate 5,Average
Baseline,0.808879,0.814469,0.800915,0.811292,0.808424,0.808796
AdversarialDebiase,0.812421,0.768374,0.823656,0.850816,0.799362,0.810926
GridSearchReduction(EO),0.807614,0.817286,0.806251,0.811669,0.813784,0.811321
GridSearchReduction(DP),0.806223,0.814597,0.802820,0.806399,0.812891,0.808586


Disparate Impact,Cross Validate 1,Cross Validate 2,Cross Validate 3,Cross Validate 4,Cross Validate 5,Average
Baseline,0.868369,0.849506,0.836084,0.900317,0.871382,0.865132
AdversarialDebiase,0.936992,1.007966,0.902299,1.036675,1.045745,0.985935
GridSearchReduction(EO),0.862968,0.860570,0.875929,0.912733,0.902493,0.882939
GridSearchReduction(DP),0.880214,0.866750,0.882383,0.896506,0.911695,0.887510


Disparate Parity,Cross Validate 1,Cross Validate 2,Cross Validate 3,Cross Validate 4,Cross Validate 5,Average
Baseline,-0.073760,-0.084710,-0.091192,-0.056697,-0.071764,-0.075625
AdversarialDebiase,-0.034925,0.004028,-0.055436,0.021853,0.024379,-0.008020
GridSearchReduction(EO),-0.076636,-0.078625,-0.069422,-0.049922,-0.054824,-0.065886
GridSearchReduction(DP),-0.066895,-0.074917,-0.065469,-0.058766,-0.049457,-0.063101


Equal Opportuniy,Cross Validate 1,Cross Validate 2,Cross Validate 3,Cross Validate 4,Cross Validate 5,Average
Baseline,-0.013407,-0.037173,-0.042153,-0.016031,-0.027150,-0.027183
AdversarialDebiase,0.039821,0.053543,0.004738,0.060352,0.075964,0.046884
GridSearchReduction(EO),-0.007675,-0.028568,-0.009942,-0.011963,-0.011252,-0.013880
GridSearchReduction(DP),0.006962,-0.028031,-0.013405,-0.035404,-0.017498,-0.017475


Equalized Odds,Cross Validate 1,Cross Validate 2,Cross Validate 3,Cross Validate 4,Cross Validate 5,Average
Baseline,-0.067385,-0.053033,-0.059067,-0.023696,-0.040905,-0.048817
AdversarialDebiase,-0.040384,0.035669,-0.030302,0.059121,0.051905,0.015202
GridSearchReduction(EO),-0.078422,-0.048766,-0.046326,-0.015344,-0.022995,-0.042371
GridSearchReduction(DP),-0.073622,-0.042601,-0.035992,-0.010670,-0.006905,-0.033958


In [16]:
#Kamiran and Calders, 2012 implementation
from aif360.algorithms.preprocessing import Reweighing
def Reweight_CV(train_set,test_set):
    #Training
    reweight=Reweighing(privileged_groups=privileged_groups,unprivileged_groups=unprivileged_groups)
    reweight.fit(train_set)
    reweight_set=reweight.transform(train_set.copy(deepcopy=True))
    reweight_test = reweight.transform(test_set.copy(deepcopy=True))

    reweight_model=RandomForestClassifier()
    reweight_model.fit(reweight_set.features,reweight_set.labels.ravel(),sample_weight=train_set.instance_weights)
    #Testing
    pred=reweight_model.predict(reweight_test.features)
    test_pred=test_set.copy(deepcopy=True)
    test_pred.labels=pred
    return test_pred

cross_validation("Reweight",Reweight_CV)

Accuracy,Cross Validate 1,Cross Validate 2,Cross Validate 3,Cross Validate 4,Cross Validate 5,Average
Baseline,0.777105,0.777105,0.772053,0.772802,0.775592,0.774931
AdversarialDebiase,0.784319,0.779419,0.783011,0.783283,0.784781,0.782963
GridSearchReduction(EO),0.777037,0.778670,0.773142,0.769466,0.775932,0.774850
GridSearchReduction(DP),0.775675,0.777241,0.772121,0.768582,0.776817,0.774087
Reweight,0.774042,0.777513,0.772529,0.771712,0.775660,0.774291


Precision,Cross Validate 1,Cross Validate 2,Cross Validate 3,Cross Validate 4,Cross Validate 5,Average
Baseline,0.783797,0.776963,0.779619,0.779036,0.779090,0.779701
AdversarialDebiase,0.792083,0.807346,0.782687,0.772675,0.797530,0.790464
GridSearchReduction(EO),0.784398,0.777656,0.778268,0.774267,0.776708,0.778260
GridSearchReduction(DP),0.783239,0.777086,0.778681,0.775833,0.778416,0.778651
Reweight,0.782807,0.778213,0.778776,0.777337,0.776263,0.778679


Recall,Cross Validate 1,Cross Validate 2,Cross Validate 3,Cross Validate 4,Cross Validate 5,Average
Baseline,0.808879,0.814469,0.800915,0.811292,0.808424,0.808796
AdversarialDebiase,0.812421,0.768374,0.823656,0.850816,0.799362,0.810926
GridSearchReduction(EO),0.807614,0.817286,0.806251,0.811669,0.813784,0.811321
GridSearchReduction(DP),0.806223,0.814597,0.802820,0.806399,0.812891,0.808586
Reweight,0.802808,0.813188,0.803710,0.811669,0.813912,0.809057


Disparate Impact,Cross Validate 1,Cross Validate 2,Cross Validate 3,Cross Validate 4,Cross Validate 5,Average
Baseline,0.868369,0.849506,0.836084,0.900317,0.871382,0.865132
AdversarialDebiase,0.936992,1.007966,0.902299,1.036675,1.045745,0.985935
GridSearchReduction(EO),0.862968,0.860570,0.875929,0.912733,0.902493,0.882939
GridSearchReduction(DP),0.880214,0.866750,0.882383,0.896506,0.911695,0.887510
Reweight,0.870386,0.854299,0.837965,0.892148,0.873800,0.865720


Disparate Parity,Cross Validate 1,Cross Validate 2,Cross Validate 3,Cross Validate 4,Cross Validate 5,Average
Baseline,-0.073760,-0.084710,-0.091192,-0.056697,-0.071764,-0.075625
AdversarialDebiase,-0.034925,0.004028,-0.055436,0.021853,0.024379,-0.008020
GridSearchReduction(EO),-0.076636,-0.078625,-0.069422,-0.049922,-0.054824,-0.065886
GridSearchReduction(DP),-0.066895,-0.074917,-0.065469,-0.058766,-0.049457,-0.063101
Reweight,-0.072166,-0.081726,-0.090546,-0.061540,-0.071140,-0.075423


Equal Opportuniy,Cross Validate 1,Cross Validate 2,Cross Validate 3,Cross Validate 4,Cross Validate 5,Average
Baseline,-0.013407,-0.037173,-0.042153,-0.016031,-0.027150,-0.027183
AdversarialDebiase,0.039821,0.053543,0.004738,0.060352,0.075964,0.046884
GridSearchReduction(EO),-0.007675,-0.028568,-0.009942,-0.011963,-0.011252,-0.013880
GridSearchReduction(DP),0.006962,-0.028031,-0.013405,-0.035404,-0.017498,-0.017475
Reweight,-0.009131,-0.038132,-0.040383,-0.018666,-0.013785,-0.024019


Equalized Odds,Cross Validate 1,Cross Validate 2,Cross Validate 3,Cross Validate 4,Cross Validate 5,Average
Baseline,-0.067385,-0.053033,-0.059067,-0.023696,-0.040905,-0.048817
AdversarialDebiase,-0.040384,0.035669,-0.030302,0.059121,0.051905,0.015202
GridSearchReduction(EO),-0.078422,-0.048766,-0.046326,-0.015344,-0.022995,-0.042371
GridSearchReduction(DP),-0.073622,-0.042601,-0.035992,-0.010670,-0.006905,-0.033958
Reweight,-0.068985,-0.046406,-0.059337,-0.030942,-0.051826,-0.051499


In [18]:
#Feldman et al., 2015 implementation
from aif360.algorithms.preprocessing import DisparateImpactRemover

def DisparateImpactRemover_05_CV(train_set,test_set):
    #Training
    di = DisparateImpactRemover(repair_level=0.5,sensitive_attribute="Gender")
    train_di = di.fit_transform(train_set.copy(deepcopy=True))
    test_di = di.fit_transform(test_set.copy(deepcopy=True))

    di_model=RandomForestClassifier()
    di_model.fit(train_di.features,train_di.labels.ravel(),sample_weight=train_di.instance_weights)
    #Testing
    pred=di_model.predict(test_di.features)
    test_pred=test_set.copy(deepcopy=True)
    test_pred.labels=pred
    return test_pred

cross_validation("DisparateImpactRemover(0.5)",DisparateImpactRemover_05_CV)

Accuracy,Cross Validate 1,Cross Validate 2,Cross Validate 3,Cross Validate 4,Cross Validate 5,Average
Baseline,0.777105,0.777105,0.772053,0.772802,0.775592,0.774931
AdversarialDebiase,0.784319,0.779419,0.783011,0.783283,0.784781,0.782963
GridSearchReduction(EO),0.777037,0.778670,0.773142,0.769466,0.775932,0.774850
GridSearchReduction(DP),0.775675,0.777241,0.772121,0.768582,0.776817,0.774087
Reweight,0.774042,0.777513,0.772529,0.771712,0.775660,0.774291
DisparateImpactRemover(0.5),0.776288,0.771388,0.775796,0.774912,0.776409,0.774959


Precision,Cross Validate 1,Cross Validate 2,Cross Validate 3,Cross Validate 4,Cross Validate 5,Average
Baseline,0.783797,0.776963,0.779619,0.779036,0.779090,0.779701
AdversarialDebiase,0.792083,0.807346,0.782687,0.772675,0.797530,0.790464
GridSearchReduction(EO),0.784398,0.777656,0.778268,0.774267,0.776708,0.778260
GridSearchReduction(DP),0.783239,0.777086,0.778681,0.775833,0.778416,0.778651
Reweight,0.782807,0.778213,0.778776,0.777337,0.776263,0.778679
DisparateImpactRemover(0.5),0.792824,0.765541,0.791640,0.801422,0.777101,0.785706


Recall,Cross Validate 1,Cross Validate 2,Cross Validate 3,Cross Validate 4,Cross Validate 5,Average
Baseline,0.808879,0.814469,0.800915,0.811292,0.808424,0.808796
AdversarialDebiase,0.812421,0.768374,0.823656,0.850816,0.799362,0.810926
GridSearchReduction(EO),0.807614,0.817286,0.806251,0.811669,0.813784,0.811321
GridSearchReduction(DP),0.806223,0.814597,0.802820,0.806399,0.812891,0.808586
Reweight,0.802808,0.813188,0.803710,0.811669,0.813912,0.809057
DisparateImpactRemover(0.5),0.790918,0.821511,0.789226,0.777792,0.814295,0.798748


Disparate Impact,Cross Validate 1,Cross Validate 2,Cross Validate 3,Cross Validate 4,Cross Validate 5,Average
Baseline,0.868369,0.849506,0.836084,0.900317,0.871382,0.865132
AdversarialDebiase,0.936992,1.007966,0.902299,1.036675,1.045745,0.985935
GridSearchReduction(EO),0.862968,0.860570,0.875929,0.912733,0.902493,0.882939
GridSearchReduction(DP),0.880214,0.866750,0.882383,0.896506,0.911695,0.887510
Reweight,0.870386,0.854299,0.837965,0.892148,0.873800,0.865720
DisparateImpactRemover(0.5),0.945499,0.881053,0.918571,1.007550,0.922448,0.935024


Disparate Parity,Cross Validate 1,Cross Validate 2,Cross Validate 3,Cross Validate 4,Cross Validate 5,Average
Baseline,-0.073760,-0.084710,-0.091192,-0.056697,-0.071764,-0.075625
AdversarialDebiase,-0.034925,0.004028,-0.055436,0.021853,0.024379,-0.008020
GridSearchReduction(EO),-0.076636,-0.078625,-0.069422,-0.049922,-0.054824,-0.065886
GridSearchReduction(DP),-0.066895,-0.074917,-0.065469,-0.058766,-0.049457,-0.063101
Reweight,-0.072166,-0.081726,-0.090546,-0.061540,-0.071140,-0.075423
DisparateImpactRemover(0.5),-0.029365,-0.068394,-0.043724,0.003973,-0.043553,-0.036213


Equal Opportuniy,Cross Validate 1,Cross Validate 2,Cross Validate 3,Cross Validate 4,Cross Validate 5,Average
Baseline,-0.013407,-0.037173,-0.042153,-0.016031,-0.027150,-0.027183
AdversarialDebiase,0.039821,0.053543,0.004738,0.060352,0.075964,0.046884
GridSearchReduction(EO),-0.007675,-0.028568,-0.009942,-0.011963,-0.011252,-0.013880
GridSearchReduction(DP),0.006962,-0.028031,-0.013405,-0.035404,-0.017498,-0.017475
Reweight,-0.009131,-0.038132,-0.040383,-0.018666,-0.013785,-0.024019
DisparateImpactRemover(0.5),0.014500,-0.009862,-0.008458,0.030769,0.012184,0.007827


Equalized Odds,Cross Validate 1,Cross Validate 2,Cross Validate 3,Cross Validate 4,Cross Validate 5,Average
Baseline,-0.067385,-0.053033,-0.059067,-0.023696,-0.040905,-0.048817
AdversarialDebiase,-0.040384,0.035669,-0.030302,0.059121,0.051905,0.015202
GridSearchReduction(EO),-0.078422,-0.048766,-0.046326,-0.015344,-0.022995,-0.042371
GridSearchReduction(DP),-0.073622,-0.042601,-0.035992,-0.010670,-0.006905,-0.033958
Reweight,-0.068985,-0.046406,-0.059337,-0.030942,-0.051826,-0.051499
DisparateImpactRemover(0.5),-0.007137,-0.048189,0.002178,0.051529,-0.022523,-0.004828


In [20]:
#Feldman et al., 2015 implementation
def DisparateImpactRemover_07_CV(train_set,test_set):
    #Training
    di = DisparateImpactRemover(repair_level=0.7,sensitive_attribute="Gender")
    train_di = di.fit_transform(train_set.copy(deepcopy=True))
    test_di = di.fit_transform(test_set.copy(deepcopy=True))

    di_model=RandomForestClassifier()
    di_model.fit(train_di.features,train_di.labels.ravel(),sample_weight=train_di.instance_weights)
    #Testing
    pred=di_model.predict(test_di.features)
    test_pred=test_set.copy(deepcopy=True)
    test_pred.labels=pred
    return test_pred

cross_validation("DisparateImpactRemover(0.7)",DisparateImpactRemover_07_CV)

Accuracy,Cross Validate 1,Cross Validate 2,Cross Validate 3,Cross Validate 4,Cross Validate 5,Average
Baseline,0.777105,0.777105,0.772053,0.772802,0.775592,0.774931
AdversarialDebiase,0.784319,0.779419,0.783011,0.783283,0.784781,0.782963
GridSearchReduction(EO),0.777037,0.778670,0.773142,0.769466,0.775932,0.774850
GridSearchReduction(DP),0.775675,0.777241,0.772121,0.768582,0.776817,0.774087
Reweight,0.774042,0.777513,0.772529,0.771712,0.775660,0.774291
DisparateImpactRemover(0.5),0.776288,0.771388,0.775796,0.774912,0.776409,0.774959
DisparateImpactRemover(0.7),0.764786,0.767781,0.764225,0.752246,0.773959,0.764599


Precision,Cross Validate 1,Cross Validate 2,Cross Validate 3,Cross Validate 4,Cross Validate 5,Average
Baseline,0.783797,0.776963,0.779619,0.779036,0.779090,0.779701
AdversarialDebiase,0.792083,0.807346,0.782687,0.772675,0.797530,0.790464
GridSearchReduction(EO),0.784398,0.777656,0.778268,0.774267,0.776708,0.778260
GridSearchReduction(DP),0.783239,0.777086,0.778681,0.775833,0.778416,0.778651
Reweight,0.782807,0.778213,0.778776,0.777337,0.776263,0.778679
DisparateImpactRemover(0.5),0.792824,0.765541,0.791640,0.801422,0.777101,0.785706
DisparateImpactRemover(0.7),0.811450,0.764557,0.816914,0.834415,0.774976,0.800462


Recall,Cross Validate 1,Cross Validate 2,Cross Validate 3,Cross Validate 4,Cross Validate 5,Average
Baseline,0.808879,0.814469,0.800915,0.811292,0.808424,0.808796
AdversarialDebiase,0.812421,0.768374,0.823656,0.850816,0.799362,0.810926
GridSearchReduction(EO),0.807614,0.817286,0.806251,0.811669,0.813784,0.811321
GridSearchReduction(DP),0.806223,0.814597,0.802820,0.806399,0.812891,0.808586
Reweight,0.802808,0.813188,0.803710,0.811669,0.813912,0.809057
DisparateImpactRemover(0.5),0.790918,0.821511,0.789226,0.777792,0.814295,0.798748
DisparateImpactRemover(0.7),0.733241,0.813700,0.721636,0.677792,0.811870,0.751648


Disparate Impact,Cross Validate 1,Cross Validate 2,Cross Validate 3,Cross Validate 4,Cross Validate 5,Average
Baseline,0.868369,0.849506,0.836084,0.900317,0.871382,0.865132
AdversarialDebiase,0.936992,1.007966,0.902299,1.036675,1.045745,0.985935
GridSearchReduction(EO),0.862968,0.860570,0.875929,0.912733,0.902493,0.882939
GridSearchReduction(DP),0.880214,0.866750,0.882383,0.896506,0.911695,0.887510
Reweight,0.870386,0.854299,0.837965,0.892148,0.873800,0.865720
DisparateImpactRemover(0.5),0.945499,0.881053,0.918571,1.007550,0.922448,0.935024
DisparateImpactRemover(0.7),1.049234,0.892625,1.026410,1.241399,0.920677,1.026069


Disparate Parity,Cross Validate 1,Cross Validate 2,Cross Validate 3,Cross Validate 4,Cross Validate 5,Average
Baseline,-0.073760,-0.084710,-0.091192,-0.056697,-0.071764,-0.075625
AdversarialDebiase,-0.034925,0.004028,-0.055436,0.021853,0.024379,-0.008020
GridSearchReduction(EO),-0.076636,-0.078625,-0.069422,-0.049922,-0.054824,-0.065886
GridSearchReduction(DP),-0.066895,-0.074917,-0.065469,-0.058766,-0.049457,-0.063101
Reweight,-0.072166,-0.081726,-0.090546,-0.061540,-0.071140,-0.075423
DisparateImpactRemover(0.5),-0.029365,-0.068394,-0.043724,0.003973,-0.043553,-0.036213
DisparateImpactRemover(0.7),0.023858,-0.061184,0.012477,0.104662,-0.044542,0.007054


Equal Opportuniy,Cross Validate 1,Cross Validate 2,Cross Validate 3,Cross Validate 4,Cross Validate 5,Average
Baseline,-0.013407,-0.037173,-0.042153,-0.016031,-0.027150,-0.027183
AdversarialDebiase,0.039821,0.053543,0.004738,0.060352,0.075964,0.046884
GridSearchReduction(EO),-0.007675,-0.028568,-0.009942,-0.011963,-0.011252,-0.013880
GridSearchReduction(DP),0.006962,-0.028031,-0.013405,-0.035404,-0.017498,-0.017475
Reweight,-0.009131,-0.038132,-0.040383,-0.018666,-0.013785,-0.024019
DisparateImpactRemover(0.5),0.014500,-0.009862,-0.008458,0.030769,0.012184,0.007827
DisparateImpactRemover(0.7),0.093494,0.003074,0.056111,0.159463,0.007561,0.063941


Equalized Odds,Cross Validate 1,Cross Validate 2,Cross Validate 3,Cross Validate 4,Cross Validate 5,Average
Baseline,-0.067385,-0.053033,-0.059067,-0.023696,-0.040905,-0.048817
AdversarialDebiase,-0.040384,0.035669,-0.030302,0.059121,0.051905,0.015202
GridSearchReduction(EO),-0.078422,-0.048766,-0.046326,-0.015344,-0.022995,-0.042371
GridSearchReduction(DP),-0.073622,-0.042601,-0.035992,-0.010670,-0.006905,-0.033958
Reweight,-0.068985,-0.046406,-0.059337,-0.030942,-0.051826,-0.051499
DisparateImpactRemover(0.5),-0.007137,-0.048189,0.002178,0.051529,-0.022523,-0.004828
DisparateImpactRemover(0.7),0.019955,-0.046930,0.049027,0.122136,-0.020882,0.024661


In [21]:
#Feldman et al., 2015 implementation
def DisparateImpactRemover_10_CV(train_set,test_set):
    #Training
    di = DisparateImpactRemover(repair_level=1.0,sensitive_attribute="Gender")
    train_di = di.fit_transform(train_set.copy(deepcopy=True))
    test_di = di.fit_transform(test_set.copy(deepcopy=True))

    di_model=RandomForestClassifier()
    di_model.fit(train_di.features,train_di.labels.ravel(),sample_weight=train_di.instance_weights)
    #Testing
    pred=di_model.predict(test_di.features)
    test_pred=test_set.copy(deepcopy=True)
    test_pred.labels=pred
    return test_pred

cross_validation("DisparateImpactRemover(1.0)",DisparateImpactRemover_10_CV)

Accuracy,Cross Validate 1,Cross Validate 2,Cross Validate 3,Cross Validate 4,Cross Validate 5,Average
Baseline,0.777105,0.777105,0.772053,0.772802,0.775592,0.774931
AdversarialDebiase,0.784319,0.779419,0.783011,0.783283,0.784781,0.782963
GridSearchReduction(EO),0.777037,0.778670,0.773142,0.769466,0.775932,0.774850
GridSearchReduction(DP),0.775675,0.777241,0.772121,0.768582,0.776817,0.774087
Reweight,0.774042,0.777513,0.772529,0.771712,0.775660,0.774291
DisparateImpactRemover(0.5),0.776288,0.771388,0.775796,0.774912,0.776409,0.774959
DisparateImpactRemover(0.7),0.764786,0.767781,0.764225,0.752246,0.773959,0.764599
DisparateImpactRemover(1.0),0.765262,0.766419,0.764089,0.741968,0.771372,0.761822


Precision,Cross Validate 1,Cross Validate 2,Cross Validate 3,Cross Validate 4,Cross Validate 5,Average
Baseline,0.783797,0.776963,0.779619,0.779036,0.779090,0.779701
AdversarialDebiase,0.792083,0.807346,0.782687,0.772675,0.797530,0.790464
GridSearchReduction(EO),0.784398,0.777656,0.778268,0.774267,0.776708,0.778260
GridSearchReduction(DP),0.783239,0.777086,0.778681,0.775833,0.778416,0.778651
Reweight,0.782807,0.778213,0.778776,0.777337,0.776263,0.778679
DisparateImpactRemover(0.5),0.792824,0.765541,0.791640,0.801422,0.777101,0.785706
DisparateImpactRemover(0.7),0.811450,0.764557,0.816914,0.834415,0.774976,0.800462
DisparateImpactRemover(1.0),0.817948,0.763799,0.819064,0.836636,0.773728,0.802235


Recall,Cross Validate 1,Cross Validate 2,Cross Validate 3,Cross Validate 4,Cross Validate 5,Average
Baseline,0.808879,0.814469,0.800915,0.811292,0.808424,0.808796
AdversarialDebiase,0.812421,0.768374,0.823656,0.850816,0.799362,0.810926
GridSearchReduction(EO),0.807614,0.817286,0.806251,0.811669,0.813784,0.811321
GridSearchReduction(DP),0.806223,0.814597,0.802820,0.806399,0.812891,0.808586
Reweight,0.802808,0.813188,0.803710,0.811669,0.813912,0.809057
DisparateImpactRemover(0.5),0.790918,0.821511,0.789226,0.777792,0.814295,0.798748
DisparateImpactRemover(0.7),0.733241,0.813700,0.721636,0.677792,0.811870,0.751648
DisparateImpactRemover(1.0),0.725145,0.811524,0.718333,0.651568,0.807403,0.742795


Disparate Impact,Cross Validate 1,Cross Validate 2,Cross Validate 3,Cross Validate 4,Cross Validate 5,Average
Baseline,0.868369,0.849506,0.836084,0.900317,0.871382,0.865132
AdversarialDebiase,0.936992,1.007966,0.902299,1.036675,1.045745,0.985935
GridSearchReduction(EO),0.862968,0.860570,0.875929,0.912733,0.902493,0.882939
GridSearchReduction(DP),0.880214,0.866750,0.882383,0.896506,0.911695,0.887510
Reweight,0.870386,0.854299,0.837965,0.892148,0.873800,0.865720
DisparateImpactRemover(0.5),0.945499,0.881053,0.918571,1.007550,0.922448,0.935024
DisparateImpactRemover(0.7),1.049234,0.892625,1.026410,1.241399,0.920677,1.026069
DisparateImpactRemover(1.0),1.071036,0.888531,1.051063,1.315805,0.936630,1.052613


Disparate Parity,Cross Validate 1,Cross Validate 2,Cross Validate 3,Cross Validate 4,Cross Validate 5,Average
Baseline,-0.073760,-0.084710,-0.091192,-0.056697,-0.071764,-0.075625
AdversarialDebiase,-0.034925,0.004028,-0.055436,0.021853,0.024379,-0.008020
GridSearchReduction(EO),-0.076636,-0.078625,-0.069422,-0.049922,-0.054824,-0.065886
GridSearchReduction(DP),-0.066895,-0.074917,-0.065469,-0.058766,-0.049457,-0.063101
Reweight,-0.072166,-0.081726,-0.090546,-0.061540,-0.071140,-0.075423
DisparateImpactRemover(0.5),-0.029365,-0.068394,-0.043724,0.003973,-0.043553,-0.036213
DisparateImpactRemover(0.7),0.023858,-0.061184,0.012477,0.104662,-0.044542,0.007054
DisparateImpactRemover(1.0),0.033722,-0.063427,0.023912,0.130629,-0.035409,0.017885


Equal Opportuniy,Cross Validate 1,Cross Validate 2,Cross Validate 3,Cross Validate 4,Cross Validate 5,Average
Baseline,-0.013407,-0.037173,-0.042153,-0.016031,-0.027150,-0.027183
AdversarialDebiase,0.039821,0.053543,0.004738,0.060352,0.075964,0.046884
GridSearchReduction(EO),-0.007675,-0.028568,-0.009942,-0.011963,-0.011252,-0.013880
GridSearchReduction(DP),0.006962,-0.028031,-0.013405,-0.035404,-0.017498,-0.017475
Reweight,-0.009131,-0.038132,-0.040383,-0.018666,-0.013785,-0.024019
DisparateImpactRemover(0.5),0.014500,-0.009862,-0.008458,0.030769,0.012184,0.007827
DisparateImpactRemover(0.7),0.093494,0.003074,0.056111,0.159463,0.007561,0.063941
DisparateImpactRemover(1.0),0.099927,-0.001572,0.073815,0.182884,0.019489,0.074908


Equalized Odds,Cross Validate 1,Cross Validate 2,Cross Validate 3,Cross Validate 4,Cross Validate 5,Average
Baseline,-0.067385,-0.053033,-0.059067,-0.023696,-0.040905,-0.048817
AdversarialDebiase,-0.040384,0.035669,-0.030302,0.059121,0.051905,0.015202
GridSearchReduction(EO),-0.078422,-0.048766,-0.046326,-0.015344,-0.022995,-0.042371
GridSearchReduction(DP),-0.073622,-0.042601,-0.035992,-0.010670,-0.006905,-0.033958
Reweight,-0.068985,-0.046406,-0.059337,-0.030942,-0.051826,-0.051499
DisparateImpactRemover(0.5),-0.007137,-0.048189,0.002178,0.051529,-0.022523,-0.004828
DisparateImpactRemover(0.7),0.019955,-0.046930,0.049027,0.122136,-0.020882,0.024661
DisparateImpactRemover(1.0),0.033365,-0.047402,0.054939,0.148219,-0.014882,0.034848


In [23]:
#Pleiss et al., 2017 implementation
from aif360.algorithms.postprocessing import CalibratedEqOddsPostprocessing

def CalibratedEqOddsPostprocessing_CV(train_set,test_set):
    #Training
    ceop_model=RandomForestClassifier()
    ceop_model.fit(train_set.features,train_set.labels.ravel())

    train_eop_set=train_set.copy(deepcopy=True)
    train_eop_set.scores=ceop_model.predict(train_set.features)

    ceop=CalibratedEqOddsPostprocessing(unprivileged_groups=unprivileged_groups,privileged_groups=privileged_groups)
    ceop.fit(train_set,train_eop_set)
    #Testing
    pred=ceop_model.predict(test_set.features)
    test_pred=test_set.copy(deepcopy=True)
    test_pred.scores=pred
    test_pred=ceop.predict(test_pred)
    return test_pred

cross_validation("CalibratedEqOddsPostprocessing",CalibratedEqOddsPostprocessing_CV)

Accuracy,Cross Validate 1,Cross Validate 2,Cross Validate 3,Cross Validate 4,Cross Validate 5,Average
Baseline,0.777105,0.777105,0.772053,0.772802,0.775592,0.774931
AdversarialDebiase,0.784319,0.779419,0.783011,0.783283,0.784781,0.782963
GridSearchReduction(EO),0.777037,0.778670,0.773142,0.769466,0.775932,0.774850
GridSearchReduction(DP),0.775675,0.777241,0.772121,0.768582,0.776817,0.774087
Reweight,0.774042,0.777513,0.772529,0.771712,0.775660,0.774291
DisparateImpactRemover(0.5),0.776288,0.771388,0.775796,0.774912,0.776409,0.774959
DisparateImpactRemover(0.7),0.764786,0.767781,0.764225,0.752246,0.773959,0.764599
DisparateImpactRemover(1.0),0.765262,0.766419,0.764089,0.741968,0.771372,0.761822
CalibratedEqOddsPostprocessing,0.774587,0.777990,0.772733,0.771372,0.776613,0.774659


Precision,Cross Validate 1,Cross Validate 2,Cross Validate 3,Cross Validate 4,Cross Validate 5,Average
Baseline,0.783797,0.776963,0.779619,0.779036,0.779090,0.779701
AdversarialDebiase,0.792083,0.807346,0.782687,0.772675,0.797530,0.790464
GridSearchReduction(EO),0.784398,0.777656,0.778268,0.774267,0.776708,0.778260
GridSearchReduction(DP),0.783239,0.777086,0.778681,0.775833,0.778416,0.778651
Reweight,0.782807,0.778213,0.778776,0.777337,0.776263,0.778679
DisparateImpactRemover(0.5),0.792824,0.765541,0.791640,0.801422,0.777101,0.785706
DisparateImpactRemover(0.7),0.811450,0.764557,0.816914,0.834415,0.774976,0.800462
DisparateImpactRemover(1.0),0.817948,0.763799,0.819064,0.836636,0.773728,0.802235
CalibratedEqOddsPostprocessing,0.782882,0.777047,0.778378,0.777872,0.777995,0.778835


Recall,Cross Validate 1,Cross Validate 2,Cross Validate 3,Cross Validate 4,Cross Validate 5,Average
Baseline,0.808879,0.814469,0.800915,0.811292,0.808424,0.808796
AdversarialDebiase,0.812421,0.768374,0.823656,0.850816,0.799362,0.810926
GridSearchReduction(EO),0.807614,0.817286,0.806251,0.811669,0.813784,0.811321
GridSearchReduction(DP),0.806223,0.814597,0.802820,0.806399,0.812891,0.808586
Reweight,0.802808,0.813188,0.803710,0.811669,0.813912,0.809057
DisparateImpactRemover(0.5),0.790918,0.821511,0.789226,0.777792,0.814295,0.798748
DisparateImpactRemover(0.7),0.733241,0.813700,0.721636,0.677792,0.811870,0.751648
DisparateImpactRemover(1.0),0.725145,0.811524,0.718333,0.651568,0.807403,0.742795
CalibratedEqOddsPostprocessing,0.804073,0.816645,0.804980,0.809787,0.813146,0.809726


Disparate Impact,Cross Validate 1,Cross Validate 2,Cross Validate 3,Cross Validate 4,Cross Validate 5,Average
Baseline,0.868369,0.849506,0.836084,0.900317,0.871382,0.865132
AdversarialDebiase,0.936992,1.007966,0.902299,1.036675,1.045745,0.985935
GridSearchReduction(EO),0.862968,0.860570,0.875929,0.912733,0.902493,0.882939
GridSearchReduction(DP),0.880214,0.866750,0.882383,0.896506,0.911695,0.887510
Reweight,0.870386,0.854299,0.837965,0.892148,0.873800,0.865720
DisparateImpactRemover(0.5),0.945499,0.881053,0.918571,1.007550,0.922448,0.935024
DisparateImpactRemover(0.7),1.049234,0.892625,1.026410,1.241399,0.920677,1.026069
DisparateImpactRemover(1.0),1.071036,0.888531,1.051063,1.315805,0.936630,1.052613
CalibratedEqOddsPostprocessing,0.865229,0.864394,0.820395,0.889362,0.860733,0.860023


Disparate Parity,Cross Validate 1,Cross Validate 2,Cross Validate 3,Cross Validate 4,Cross Validate 5,Average
Baseline,-0.073760,-0.084710,-0.091192,-0.056697,-0.071764,-0.075625
AdversarialDebiase,-0.034925,0.004028,-0.055436,0.021853,0.024379,-0.008020
GridSearchReduction(EO),-0.076636,-0.078625,-0.069422,-0.049922,-0.054824,-0.065886
GridSearchReduction(DP),-0.066895,-0.074917,-0.065469,-0.058766,-0.049457,-0.063101
Reweight,-0.072166,-0.081726,-0.090546,-0.061540,-0.071140,-0.075423
DisparateImpactRemover(0.5),-0.029365,-0.068394,-0.043724,0.003973,-0.043553,-0.036213
DisparateImpactRemover(0.7),0.023858,-0.061184,0.012477,0.104662,-0.044542,0.007054
DisparateImpactRemover(1.0),0.033722,-0.063427,0.023912,0.130629,-0.035409,0.017885
CalibratedEqOddsPostprocessing,-0.075175,-0.076449,-0.100692,-0.062952,-0.078324,-0.078718


Equal Opportuniy,Cross Validate 1,Cross Validate 2,Cross Validate 3,Cross Validate 4,Cross Validate 5,Average
Baseline,-0.013407,-0.037173,-0.042153,-0.016031,-0.027150,-0.027183
AdversarialDebiase,0.039821,0.053543,0.004738,0.060352,0.075964,0.046884
GridSearchReduction(EO),-0.007675,-0.028568,-0.009942,-0.011963,-0.011252,-0.013880
GridSearchReduction(DP),0.006962,-0.028031,-0.013405,-0.035404,-0.017498,-0.017475
Reweight,-0.009131,-0.038132,-0.040383,-0.018666,-0.013785,-0.024019
DisparateImpactRemover(0.5),0.014500,-0.009862,-0.008458,0.030769,0.012184,0.007827
DisparateImpactRemover(0.7),0.093494,0.003074,0.056111,0.159463,0.007561,0.063941
DisparateImpactRemover(1.0),0.099927,-0.001572,0.073815,0.182884,0.019489,0.074908
CalibratedEqOddsPostprocessing,-0.012671,-0.020929,-0.051198,-0.023367,-0.039348,-0.029503


Equalized Odds,Cross Validate 1,Cross Validate 2,Cross Validate 3,Cross Validate 4,Cross Validate 5,Average
Baseline,-0.067385,-0.053033,-0.059067,-0.023696,-0.040905,-0.048817
AdversarialDebiase,-0.040384,0.035669,-0.030302,0.059121,0.051905,0.015202
GridSearchReduction(EO),-0.078422,-0.048766,-0.046326,-0.015344,-0.022995,-0.042371
GridSearchReduction(DP),-0.073622,-0.042601,-0.035992,-0.010670,-0.006905,-0.033958
Reweight,-0.068985,-0.046406,-0.059337,-0.030942,-0.051826,-0.051499
DisparateImpactRemover(0.5),-0.007137,-0.048189,0.002178,0.051529,-0.022523,-0.004828
DisparateImpactRemover(0.7),0.019955,-0.046930,0.049027,0.122136,-0.020882,0.024661
DisparateImpactRemover(1.0),0.033365,-0.047402,0.054939,0.148219,-0.014882,0.034848
CalibratedEqOddsPostprocessing,-0.071383,-0.051607,-0.068878,-0.029330,-0.042208,-0.052681


In [25]:
#Kamiran et al., 2012 implementation
from aif360.algorithms.postprocessing import RejectOptionClassification

def RejectOptionClassification_EO_CV(train_set,test_set):
    #Training
    roc_EO_model=RandomForestClassifier()
    roc_EO_model.fit(train_set.features,train_set.labels.ravel())

    train_roc_EO_set=train_set.copy(deepcopy=True)
    train_roc_EO_set.scores=roc_EO_model.predict(train_set.features)

    roc_EO = RejectOptionClassification(unprivileged_groups=unprivileged_groups, 
                                    privileged_groups=privileged_groups, 
                                    metric_name="Equal opportunity difference")
    # with train_set.temporarily_ignore('labels', 'scores'):
    #     if train_set != train_roc_EO_set:
    #         display(train_set)
    #         display(train_roc_EO_set)
    roc_EO.fit(train_set, train_roc_EO_set)
    #Testing
    pred=roc_EO_model.predict(test_set.features)
    test_pred=test_set.copy(deepcopy=True)
    test_pred.scores=pred
    test_pred=roc_EO.predict(test_pred)
    return test_pred

cross_validation("RejectOptionClassification(EO)",RejectOptionClassification_EO_CV)

Accuracy,Cross Validate 1,Cross Validate 2,Cross Validate 3,Cross Validate 4,Cross Validate 5,Average
Baseline,0.777105,0.777105,0.772053,0.772802,0.775592,0.774931
AdversarialDebiase,0.784319,0.779419,0.783011,0.783283,0.784781,0.782963
GridSearchReduction(EO),0.777037,0.778670,0.773142,0.769466,0.775932,0.774850
GridSearchReduction(DP),0.775675,0.777241,0.772121,0.768582,0.776817,0.774087
Reweight,0.774042,0.777513,0.772529,0.771712,0.775660,0.774291
DisparateImpactRemover(0.5),0.776288,0.771388,0.775796,0.774912,0.776409,0.774959
DisparateImpactRemover(0.7),0.764786,0.767781,0.764225,0.752246,0.773959,0.764599
DisparateImpactRemover(1.0),0.765262,0.766419,0.764089,0.741968,0.771372,0.761822
CalibratedEqOddsPostprocessing,0.774587,0.777990,0.772733,0.771372,0.776613,0.774659
RejectOptionClassification(EO),0.773838,0.775607,0.774299,0.768718,0.774980,0.773488


Precision,Cross Validate 1,Cross Validate 2,Cross Validate 3,Cross Validate 4,Cross Validate 5,Average
Baseline,0.783797,0.776963,0.779619,0.779036,0.779090,0.779701
AdversarialDebiase,0.792083,0.807346,0.782687,0.772675,0.797530,0.790464
GridSearchReduction(EO),0.784398,0.777656,0.778268,0.774267,0.776708,0.778260
GridSearchReduction(DP),0.783239,0.777086,0.778681,0.775833,0.778416,0.778651
Reweight,0.782807,0.778213,0.778776,0.777337,0.776263,0.778679
DisparateImpactRemover(0.5),0.792824,0.765541,0.791640,0.801422,0.777101,0.785706
DisparateImpactRemover(0.7),0.811450,0.764557,0.816914,0.834415,0.774976,0.800462
DisparateImpactRemover(1.0),0.817948,0.763799,0.819064,0.836636,0.773728,0.802235
CalibratedEqOddsPostprocessing,0.782882,0.777047,0.778378,0.777872,0.777995,0.778835
RejectOptionClassification(EO),0.780032,0.775082,0.779345,0.776354,0.776800,0.777523


Recall,Cross Validate 1,Cross Validate 2,Cross Validate 3,Cross Validate 4,Cross Validate 5,Average
Baseline,0.808879,0.814469,0.800915,0.811292,0.808424,0.808796
AdversarialDebiase,0.812421,0.768374,0.823656,0.850816,0.799362,0.810926
GridSearchReduction(EO),0.807614,0.817286,0.806251,0.811669,0.813784,0.811321
GridSearchReduction(DP),0.806223,0.814597,0.802820,0.806399,0.812891,0.808586
Reweight,0.802808,0.813188,0.803710,0.811669,0.813912,0.809057
DisparateImpactRemover(0.5),0.790918,0.821511,0.789226,0.777792,0.814295,0.798748
DisparateImpactRemover(0.7),0.733241,0.813700,0.721636,0.677792,0.811870,0.751648
DisparateImpactRemover(1.0),0.725145,0.811524,0.718333,0.651568,0.807403,0.742795
CalibratedEqOddsPostprocessing,0.804073,0.816645,0.804980,0.809787,0.813146,0.809726
RejectOptionClassification(EO),0.807361,0.814085,0.807267,0.805772,0.811104,0.809118


Disparate Impact,Cross Validate 1,Cross Validate 2,Cross Validate 3,Cross Validate 4,Cross Validate 5,Average
Baseline,0.868369,0.849506,0.836084,0.900317,0.871382,0.865132
AdversarialDebiase,0.936992,1.007966,0.902299,1.036675,1.045745,0.985935
GridSearchReduction(EO),0.862968,0.860570,0.875929,0.912733,0.902493,0.882939
GridSearchReduction(DP),0.880214,0.866750,0.882383,0.896506,0.911695,0.887510
Reweight,0.870386,0.854299,0.837965,0.892148,0.873800,0.865720
DisparateImpactRemover(0.5),0.945499,0.881053,0.918571,1.007550,0.922448,0.935024
DisparateImpactRemover(0.7),1.049234,0.892625,1.026410,1.241399,0.920677,1.026069
DisparateImpactRemover(1.0),1.071036,0.888531,1.051063,1.315805,0.936630,1.052613
CalibratedEqOddsPostprocessing,0.865229,0.864394,0.820395,0.889362,0.860733,0.860023
RejectOptionClassification(EO),0.856270,0.849658,0.842559,0.896002,0.867630,0.862424


Disparate Parity,Cross Validate 1,Cross Validate 2,Cross Validate 3,Cross Validate 4,Cross Validate 5,Average
Baseline,-0.073760,-0.084710,-0.091192,-0.056697,-0.071764,-0.075625
AdversarialDebiase,-0.034925,0.004028,-0.055436,0.021853,0.024379,-0.008020
GridSearchReduction(EO),-0.076636,-0.078625,-0.069422,-0.049922,-0.054824,-0.065886
GridSearchReduction(DP),-0.066895,-0.074917,-0.065469,-0.058766,-0.049457,-0.063101
Reweight,-0.072166,-0.081726,-0.090546,-0.061540,-0.071140,-0.075423
DisparateImpactRemover(0.5),-0.029365,-0.068394,-0.043724,0.003973,-0.043553,-0.036213
DisparateImpactRemover(0.7),0.023858,-0.061184,0.012477,0.104662,-0.044542,0.007054
DisparateImpactRemover(1.0),0.033722,-0.063427,0.023912,0.130629,-0.035409,0.017885
CalibratedEqOddsPostprocessing,-0.075175,-0.076449,-0.100692,-0.062952,-0.078324,-0.078718
RejectOptionClassification(EO),-0.080844,-0.084789,-0.088278,-0.058969,-0.074339,-0.077444


Equal Opportuniy,Cross Validate 1,Cross Validate 2,Cross Validate 3,Cross Validate 4,Cross Validate 5,Average
Baseline,-0.013407,-0.037173,-0.042153,-0.016031,-0.027150,-0.027183
AdversarialDebiase,0.039821,0.053543,0.004738,0.060352,0.075964,0.046884
GridSearchReduction(EO),-0.007675,-0.028568,-0.009942,-0.011963,-0.011252,-0.013880
GridSearchReduction(DP),0.006962,-0.028031,-0.013405,-0.035404,-0.017498,-0.017475
Reweight,-0.009131,-0.038132,-0.040383,-0.018666,-0.013785,-0.024019
DisparateImpactRemover(0.5),0.014500,-0.009862,-0.008458,0.030769,0.012184,0.007827
DisparateImpactRemover(0.7),0.093494,0.003074,0.056111,0.159463,0.007561,0.063941
DisparateImpactRemover(1.0),0.099927,-0.001572,0.073815,0.182884,0.019489,0.074908
CalibratedEqOddsPostprocessing,-0.012671,-0.020929,-0.051198,-0.023367,-0.039348,-0.029503
RejectOptionClassification(EO),-0.022752,-0.039084,-0.041789,-0.021331,-0.010809,-0.027153


Equalized Odds,Cross Validate 1,Cross Validate 2,Cross Validate 3,Cross Validate 4,Cross Validate 5,Average
Baseline,-0.067385,-0.053033,-0.059067,-0.023696,-0.040905,-0.048817
AdversarialDebiase,-0.040384,0.035669,-0.030302,0.059121,0.051905,0.015202
GridSearchReduction(EO),-0.078422,-0.048766,-0.046326,-0.015344,-0.022995,-0.042371
GridSearchReduction(DP),-0.073622,-0.042601,-0.035992,-0.010670,-0.006905,-0.033958
Reweight,-0.068985,-0.046406,-0.059337,-0.030942,-0.051826,-0.051499
DisparateImpactRemover(0.5),-0.007137,-0.048189,0.002178,0.051529,-0.022523,-0.004828
DisparateImpactRemover(0.7),0.019955,-0.046930,0.049027,0.122136,-0.020882,0.024661
DisparateImpactRemover(1.0),0.033365,-0.047402,0.054939,0.148219,-0.014882,0.034848
CalibratedEqOddsPostprocessing,-0.071383,-0.051607,-0.068878,-0.029330,-0.042208,-0.052681
RejectOptionClassification(EO),-0.073130,-0.051913,-0.053330,-0.024184,-0.060725,-0.052656


In [7]:
#Kamiran et al., 2012 implementation
from aif360.algorithms.postprocessing import RejectOptionClassification

def RejectOptionClassification_DP_CV(train_set,test_set):
    #Training
    roc_EO_model=RandomForestClassifier()
    roc_EO_model.fit(train_set.features,train_set.labels.ravel())

    train_roc_EO_set=train_set.copy(deepcopy=True)
    train_roc_EO_set.scores=roc_EO_model.predict(train_set.features)

    roc_EO = RejectOptionClassification(unprivileged_groups=unprivileged_groups, 
                                    privileged_groups=privileged_groups, 
                                    metric_name="Statistical parity difference")
    roc_EO.fit(train_set, train_roc_EO_set)
    #Testing
    pred=roc_EO_model.predict(test_set.features)
    test_pred=test_set.copy(deepcopy=True)
    test_pred.scores=pred
    test_pred=roc_EO.predict(test_pred)
    return test_pred

cross_validation("RejectOptionClassification(DP)",RejectOptionClassification_DP_CV)

c:\Users\15783\source\repos\Fairness-Metrics-Research\venv\Lib\site-packages\aif360\algorithms\postprocessing\reject_option_classification.py:160: UserWarning: Unable to satisy fairness constraints
  warn("Unable to satisy fairness constraints")
c:\Users\15783\source\repos\Fairness-Metrics-Research\venv\Lib\site-packages\aif360\algorithms\postprocessing\reject_option_classification.py:160: UserWarning: Unable to satisy fairness constraints
  warn("Unable to satisy fairness constraints")
c:\Users\15783\source\repos\Fairness-Metrics-Research\venv\Lib\site-packages\aif360\algorithms\postprocessing\reject_option_classification.py:160: UserWarning: Unable to satisy fairness constraints
  warn("Unable to satisy fairness constraints")
c:\Users\15783\source\repos\Fairness-Metrics-Research\venv\Lib\site-packages\aif360\algorithms\postprocessing\reject_option_classification.py:160: UserWarning: Unable to satisy fairness constraints
  warn("Unable to satisy fairness constraints")
c:\Users\15783\s

Accuracy,Cross Validate 1,Cross Validate 2,Cross Validate 3,Cross Validate 4,Cross Validate 5,Average
Baseline,0.777105,0.777105,0.772053,0.772802,0.775592,0.774931
AdversarialDebiase,0.784319,0.779419,0.783011,0.783283,0.784781,0.782963
GridSearchReduction(EO),0.777037,0.778670,0.773142,0.769466,0.775932,0.774850
GridSearchReduction(DP),0.775675,0.777241,0.772121,0.768582,0.776817,0.774087
Reweight,0.774042,0.777513,0.772529,0.771712,0.775660,0.774291
DisparateImpactRemover(0.5),0.776288,0.771388,0.775796,0.774912,0.776409,0.774959
DisparateImpactRemover(0.7),0.764786,0.767781,0.764225,0.752246,0.773959,0.764599
DisparateImpactRemover(1.0),0.765262,0.766419,0.764089,0.741968,0.771372,0.761822
CalibratedEqOddsPostprocessing,0.774587,0.777990,0.772733,0.771372,0.776613,0.774659
RejectOptionClassification(EO),0.773838,0.775607,0.774299,0.768718,0.774980,0.773488


Precision,Cross Validate 1,Cross Validate 2,Cross Validate 3,Cross Validate 4,Cross Validate 5,Average
Baseline,0.783797,0.776963,0.779619,0.779036,0.779090,0.779701
AdversarialDebiase,0.792083,0.807346,0.782687,0.772675,0.797530,0.790464
GridSearchReduction(EO),0.784398,0.777656,0.778268,0.774267,0.776708,0.778260
GridSearchReduction(DP),0.783239,0.777086,0.778681,0.775833,0.778416,0.778651
Reweight,0.782807,0.778213,0.778776,0.777337,0.776263,0.778679
DisparateImpactRemover(0.5),0.792824,0.765541,0.791640,0.801422,0.777101,0.785706
DisparateImpactRemover(0.7),0.811450,0.764557,0.816914,0.834415,0.774976,0.800462
DisparateImpactRemover(1.0),0.817948,0.763799,0.819064,0.836636,0.773728,0.802235
CalibratedEqOddsPostprocessing,0.782882,0.777047,0.778378,0.777872,0.777995,0.778835
RejectOptionClassification(EO),0.780032,0.775082,0.779345,0.776354,0.776800,0.777523


Recall,Cross Validate 1,Cross Validate 2,Cross Validate 3,Cross Validate 4,Cross Validate 5,Average
Baseline,0.808879,0.814469,0.800915,0.811292,0.808424,0.808796
AdversarialDebiase,0.812421,0.768374,0.823656,0.850816,0.799362,0.810926
GridSearchReduction(EO),0.807614,0.817286,0.806251,0.811669,0.813784,0.811321
GridSearchReduction(DP),0.806223,0.814597,0.802820,0.806399,0.812891,0.808586
Reweight,0.802808,0.813188,0.803710,0.811669,0.813912,0.809057
DisparateImpactRemover(0.5),0.790918,0.821511,0.789226,0.777792,0.814295,0.798748
DisparateImpactRemover(0.7),0.733241,0.813700,0.721636,0.677792,0.811870,0.751648
DisparateImpactRemover(1.0),0.725145,0.811524,0.718333,0.651568,0.807403,0.742795
CalibratedEqOddsPostprocessing,0.804073,0.816645,0.804980,0.809787,0.813146,0.809726
RejectOptionClassification(EO),0.807361,0.814085,0.807267,0.805772,0.811104,0.809118


Disparate Impact,Cross Validate 1,Cross Validate 2,Cross Validate 3,Cross Validate 4,Cross Validate 5,Average
Baseline,0.868369,0.849506,0.836084,0.900317,0.871382,0.865132
AdversarialDebiase,0.936992,1.007966,0.902299,1.036675,1.045745,0.985935
GridSearchReduction(EO),0.862968,0.860570,0.875929,0.912733,0.902493,0.882939
GridSearchReduction(DP),0.880214,0.866750,0.882383,0.896506,0.911695,0.887510
Reweight,0.870386,0.854299,0.837965,0.892148,0.873800,0.865720
DisparateImpactRemover(0.5),0.945499,0.881053,0.918571,1.007550,0.922448,0.935024
DisparateImpactRemover(0.7),1.049234,0.892625,1.026410,1.241399,0.920677,1.026069
DisparateImpactRemover(1.0),1.071036,0.888531,1.051063,1.315805,0.936630,1.052613
CalibratedEqOddsPostprocessing,0.865229,0.864394,0.820395,0.889362,0.860733,0.860023
RejectOptionClassification(EO),0.856270,0.849658,0.842559,0.896002,0.867630,0.862424


Disparate Parity,Cross Validate 1,Cross Validate 2,Cross Validate 3,Cross Validate 4,Cross Validate 5,Average
Baseline,-0.073760,-0.084710,-0.091192,-0.056697,-0.071764,-0.075625
AdversarialDebiase,-0.034925,0.004028,-0.055436,0.021853,0.024379,-0.008020
GridSearchReduction(EO),-0.076636,-0.078625,-0.069422,-0.049922,-0.054824,-0.065886
GridSearchReduction(DP),-0.066895,-0.074917,-0.065469,-0.058766,-0.049457,-0.063101
Reweight,-0.072166,-0.081726,-0.090546,-0.061540,-0.071140,-0.075423
DisparateImpactRemover(0.5),-0.029365,-0.068394,-0.043724,0.003973,-0.043553,-0.036213
DisparateImpactRemover(0.7),0.023858,-0.061184,0.012477,0.104662,-0.044542,0.007054
DisparateImpactRemover(1.0),0.033722,-0.063427,0.023912,0.130629,-0.035409,0.017885
CalibratedEqOddsPostprocessing,-0.075175,-0.076449,-0.100692,-0.062952,-0.078324,-0.078718
RejectOptionClassification(EO),-0.080844,-0.084789,-0.088278,-0.058969,-0.074339,-0.077444


Equal Opportuniy,Cross Validate 1,Cross Validate 2,Cross Validate 3,Cross Validate 4,Cross Validate 5,Average
Baseline,-0.013407,-0.037173,-0.042153,-0.016031,-0.027150,-0.027183
AdversarialDebiase,0.039821,0.053543,0.004738,0.060352,0.075964,0.046884
GridSearchReduction(EO),-0.007675,-0.028568,-0.009942,-0.011963,-0.011252,-0.013880
GridSearchReduction(DP),0.006962,-0.028031,-0.013405,-0.035404,-0.017498,-0.017475
Reweight,-0.009131,-0.038132,-0.040383,-0.018666,-0.013785,-0.024019
DisparateImpactRemover(0.5),0.014500,-0.009862,-0.008458,0.030769,0.012184,0.007827
DisparateImpactRemover(0.7),0.093494,0.003074,0.056111,0.159463,0.007561,0.063941
DisparateImpactRemover(1.0),0.099927,-0.001572,0.073815,0.182884,0.019489,0.074908
CalibratedEqOddsPostprocessing,-0.012671,-0.020929,-0.051198,-0.023367,-0.039348,-0.029503
RejectOptionClassification(EO),-0.022752,-0.039084,-0.041789,-0.021331,-0.010809,-0.027153


Equalized Odds,Cross Validate 1,Cross Validate 2,Cross Validate 3,Cross Validate 4,Cross Validate 5,Average
Baseline,-0.067385,-0.053033,-0.059067,-0.023696,-0.040905,-0.048817
AdversarialDebiase,-0.040384,0.035669,-0.030302,0.059121,0.051905,0.015202
GridSearchReduction(EO),-0.078422,-0.048766,-0.046326,-0.015344,-0.022995,-0.042371
GridSearchReduction(DP),-0.073622,-0.042601,-0.035992,-0.010670,-0.006905,-0.033958
Reweight,-0.068985,-0.046406,-0.059337,-0.030942,-0.051826,-0.051499
DisparateImpactRemover(0.5),-0.007137,-0.048189,0.002178,0.051529,-0.022523,-0.004828
DisparateImpactRemover(0.7),0.019955,-0.046930,0.049027,0.122136,-0.020882,0.024661
DisparateImpactRemover(1.0),0.033365,-0.047402,0.054939,0.148219,-0.014882,0.034848
CalibratedEqOddsPostprocessing,-0.071383,-0.051607,-0.068878,-0.029330,-0.042208,-0.052681
RejectOptionClassification(EO),-0.073130,-0.051913,-0.053330,-0.024184,-0.060725,-0.052656


In [8]:
Accuracy_data.to_csv("job_accuracy.csv")
Precision_data.to_csv("job_precision.csv")
Recall_data.to_csv("job_recall.csv")
DisparateImpact_data.to_csv("job_disparate_impact.csv")
DisparateParity_data.to_csv("job_disparate_parity.csv")
EqualOpportunity_data.to_csv("job_equal_opportunity.csv")
EqualizedOdds_data.to_csv("job_equalized_odds.csv")
